In [1]:
import pandas as pd
import numpy as np

In [4]:
df = pd.read_csv("movies_metadata_clean.csv")

In [5]:
df.head()

,title,genres,runtime,vote_average,vote_count,year
0,Toy Story,"['Animation', 'Comedy', 'Family']",81.0,7.7,5415.0,1995
1,Jumanji,"['Adventure', 'Fantasy', 'Family']",104.0,6.9,2413.0,1995
2,Grumpier Old Men,"['Romance', 'Comedy']",101.0,6.5,92.0,1995
3,Waiting to Exhale,"['Comedy', 'Drama', 'Romance']",127.0,6.1,34.0,1995
4,Father of the Bride Part II,['Comedy'],106.0,5.7,173.0,1995


# Plot Description-Based recommender

1. Obtain the data required to build the model
2. Create TF-IDF vectors for the plot description (or overview) of every movie.
3. Compute the pairwise cosine similarity score of every movie
4. Write the recommender function that takes in a movie title as an argument and outpiuts movies most similar to it based on the plot.

### Preparing the Data

In [7]:
# Import the original file
orig_df = pd.read_csv("movies_metadata.csv", low_memory=False)

#Add the userful features into the cleaned dataframe
df["overview"], df['id'] = orig_df["overview"], orig_df["id"]

df.head()

,title,genres,runtime,vote_average,vote_count,year,overview,id
0,Toy Story,"['Animation', 'Comedy', 'Family']",81.0,7.7,5415.0,1995,"Led by Woody, Andy's toys live happily in his ...",862
1,Jumanji,"['Adventure', 'Fantasy', 'Family']",104.0,6.9,2413.0,1995,When siblings Judy and Peter discover an encha...,8844
2,Grumpier Old Men,"['Romance', 'Comedy']",101.0,6.5,92.0,1995,A family wedding reignites the ancient feud be...,15602
3,Waiting to Exhale,"['Comedy', 'Drama', 'Romance']",127.0,6.1,34.0,1995,"Cheated on, mistreated and stepped on, the wom...",31357
4,Father of the Bride Part II,['Comedy'],106.0,5.7,173.0,1995,Just when George Banks has recovered from his ...,11862


### Creating the TF-IDF matrix

In [19]:
#Import TfIdfVectorizer from the scikit-learn library
from sklearn.feature_extraction.text import TfidfVectorizer

#Deine a TF-IDF Vectorizer Object, Remove all english stopwords
tfidf = TfidfVectorizer(stop_words="english")

#Replace Nan with an empty string
df["overview"] = df['overview'].dropna('')

#Construct the required TF-IDF matrix by applying the fit_transform method on the overview feature
tfidf_matrix = tfidf.fit_transform(df["overview"])

#Output the shape of tfidf_matrix
tfidf_matrix.shape

(45466, 75827)

In [20]:
pd.DataFrame(tfidf_matrix) #Checking the sparse matrix 

,0
0,"(0, 17764)\t0.13483149538639247\n (0, 4388)..."
1,"(0, 14988)\t0.15233435064749146\n (0, 66484..."
2,"(0, 67150)\t0.09013326169468926\n (0, 30668..."
3,"(0, 8909)\t0.2524772113861553\n (0, 72417)\..."
4,"(0, 35872)\t0.18466136938635244\n (0, 27260..."
...,...
45461,"(0, 56405)\t0.6771266635239638\n (0, 22632)..."
45462,"(0, 63819)\t0.474848181921328\n (0, 4251)\t..."
45463,"(0, 6761)\t0.3715972018149557\n (0, 30135)\..."
45464,"(0, 20273)\t0.1677505411397102\n (0, 15394)..."


dtype('float64')

### Computing the cosine similarity score

In [21]:
#Import linear_kernel to compute the dot product
from sklearn.metrics.pairwise import linear_kernel

#Compute the cosine similarity matrix
cosine_sim = linear_kernel(tfidf_matrix.astype("float16"), tfidf_matrix.astype("float16"))

ValueError: array is too big; `arr.size * arr.dtype.itemsize` is larger than the maximum possible size.